In [13]:
from PIL import Image, ImageEnhance
import cv2
import numpy as np
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
# from deskew import determine_skew
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
import shutil
import glob
import os

def set_dpi(image, preproc_image):
    im = Image.open(image)
    im.save(preproc_image, dpi=(300,300))
    return preproc_image

def increase_contrast(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Contrast(im)
    factor = 2 #increase contrast
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def increase_sharpness(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Sharpness(im)
    factor = 2
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def convert_to_grayscale(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def convert_to_rgb(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def otsu_threshold(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # im = cv2.threshold(im, 120, 255, cv2.THRESH_BINARY, cv2.THRESH_OTSU)[1]
    im = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def remove_noise(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.fastNlMeansDenoising(im, None, 20, 7, 21)
    cv2.imwrite(preproc_image, im)
    return preproc_image

# def deskew(image):
#     try:
#         print("IMAGE --> ", image)
#         im = io.imread(image)
#         angle = determine_skew(im)
#         rotated = rotate(im, angle, resize=True) * 255
#         io.imsave(image, rotated.astype(np.uint8))
#         return image
#     except:
#         return image 

def pdf_to_images(pdf, filename):
    try:
        pdf = pdf.replace('\\', '\\\\')
        # print(pdf)
        images = convert_from_path(pdf)
        for i in range(len(images)):
            images[i].save('C:\\air_ticket\\comakeit_groundtravel\\images\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')
    except:
        print("\n PDF --> ", pdf)

def get_pdf_image():
    path = "C:\\Users\\Lenovo\\Downloads\\coMakIT-FInal data\\Business Travel Emission Management\\Ground Travel"
    #store all the file names in this list
    filelist = []
    for root, dirs, files in os.walk(path):
        for file in files:
            #append the file name to the list
            filelist.append(os.path.join(root,file))
    # print all the file names
    for file in filelist:
        filename = os.path.split(file)[-1].split('.pdf')[0]
        if ' ' in filename:
            filename = filename.replace(' ', '_')
        # print(filename)
        pdf_to_images(file, filename)

def preprocess_files():
    # print('hello')
    # combinations = ['set_dpi-convert_to_grayscale', 'set_dpi-increase_contrast-increase_sharpness', 'set_dpi-increase_contrast-convert_to_grayscale', 'set_dpi-convert_to_grayscale-remove_noise', 'set_dpi-increase_contrast-convert_to_grayscale-remove_noise', 'set_dpi-increase_contrast-convert_to_grayscale-deskew', 'set_dpi-convert_to_grayscale-remove_noise-deskew', 'set_dpi-increase_contrast-convert_to_grayscale-remove_noise-deskew',
    #                'set_dpi-convert_to_rgb']
    # combinations = ['set_dpi-increase_contrast-increase_sharpness']
    combinations = ['convert_to_rgb-set_dpi']

    for image in glob.glob('C:\\air_ticket\\comakeit_groundtravel\\images\\*.*'):
        # print('helloo')
        print("\n IMAGE --> ", image)
        # filename = os.path.split(image)[-1].split('.jpg')[0]
        # preproc_image = 'preprocess_images\\'+filename+'.jpg'
        # shutil.copy(image, preproc_image)
        for combination in combinations:
            filename = os.path.split(image)[-1].split('.jpg')[0]
            preproc_image = 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\'+filename+'.jpg'
            shutil.copy(image, preproc_image)
            methods = combination.split('-')
            for method in methods:
                if 'deskew' in method:
                    preproc_image = eval(method+'(preproc_image)')
                else:
                    preproc_image = eval(method+'(preproc_image, preproc_image)')
            # print(combination)
            image_name = 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\' + filename+'_'+combination+'.jpg'
            print(image_name)
            img = cv2.imread(preproc_image)
            cv2.imwrite(image_name, img)

            parsed_output = 'C:\\air_ticket\\comakeit_groundtravel\\images_text\\'+filename+'_4_'+combination
            command = 'tesseract {} {} -l eng --psm 4'.format(image_name, parsed_output)
            os.system(command)

            parsed_output = 'C:\\air_ticket\\comakeit_groundtravel\\images_text\\'+filename+'_3_'+combination
            command = 'tesseract {} {} -l eng --psm 3'.format(image_name, parsed_output)
            os.system(command)

# get_pdf_image()
preprocess_files()


 IMAGE -->  C:\air_ticket\comakeit_groundtravel\images\FB2324-007951_page0.jpg
C:\air_ticket\comakeit_groundtravel\preprocess_images\FB2324-007951_page0_convert_to_rgb-set_dpi.jpg

 IMAGE -->  C:\air_ticket\comakeit_groundtravel\images\FB2324-007951_page1.jpg
C:\air_ticket\comakeit_groundtravel\preprocess_images\FB2324-007951_page1_convert_to_rgb-set_dpi.jpg

 IMAGE -->  C:\air_ticket\comakeit_groundtravel\images\FC2223-052798_page0.jpg
C:\air_ticket\comakeit_groundtravel\preprocess_images\FC2223-052798_page0_convert_to_rgb-set_dpi.jpg

 IMAGE -->  C:\air_ticket\comakeit_groundtravel\images\FC2223-052798_page1.jpg
C:\air_ticket\comakeit_groundtravel\preprocess_images\FC2223-052798_page1_convert_to_rgb-set_dpi.jpg

 IMAGE -->  C:\air_ticket\comakeit_groundtravel\images\FC2223-052799_page0.jpg
C:\air_ticket\comakeit_groundtravel\preprocess_images\FC2223-052799_page0_convert_to_rgb-set_dpi.jpg

 IMAGE -->  C:\air_ticket\comakeit_groundtravel\images\FC2223-052799_page1.jpg
C:\air_ticket\c

hello


In [19]:
import cv2
import os
import re
from prettyprinter import pprint
from PIL import Image, ImageEnhance
import glob

class DelimiterComakeitGroundTravel:
    # data = []
    # os.environ['TESSERACT'] = "C:\\Program Files (x86)\\tesseract\\bin"
    # os.environ['TESSDATA_PREFIX'] = "C:\\Program Files (x86)\\tesseract\\share\\tessdata"
    # os.environ['TESSERACT'] = "/var/www/html/image-processing-deployed"
    # os.environ['TESSDATA_PREFIX'] = "/var/www/html/image-processing-deployed/tessdata" 
    # os.environ['TESSERACT'] = "/var/www/html/image-processing-deployed"
    # os.environ['TESSDATA_PREFIX'] = "/var/www/html/image-processing-deployed/tessdata" 
    # os.environ['TESSERACT'] = "C:\\Program Files (x86)\\tesseract\\bin"

    def __init__(self, path, images):
        self.path_ = path 
        self.process_images = images 

    def get_box_file(self, image, output_path):
        command = 'tesseract {} {} batch.nochop makebox'.format(image, output_path)
        os.system(command)

    def get_parsed_output(self, image, output_file):
        output_file_4 = output_file + "_4"
        command = 'tesseract {} {} -l eng --psm 4'.format(image, output_file_4)
        os.system(command)
        return [output_file_4 + ".txt"]
    
    def split_filename(self, image):
        filename = os.path.split(image)[-1].split('.jpg')[0]
        return filename

    def store_separate_lines(self, box_file):
        '''
        Method to separate lines along with box cooridnates
        into nested list.
        '''

        text = ''
        with open(box_file, 'r', encoding='utf-8') as fp:
            text = fp.read()

        text = text.splitlines()

        ch_coords = []

        for ch in text:
            ch_split = ch.split()
            ch_coords.append(ch_split)

        lines = []
        line = []
        max_diff = [1]

        for i in range(len(ch_coords)-1):
            c = ch_coords[i][0]

            if int(ch_coords[i][1]) <= int(ch_coords[i+1][1]) or int(ch_coords[i][1]) - int(ch_coords[i+1][1]) in max_diff:
                line.append(ch_coords[i])
            else:
                line.append(ch_coords[i])
                lines.append(line)
                line = []

        return lines

    def add_delimiters(self, image, filename):
        img = cv2.imread(image)
        full_path = os.path.join(self.path_, "delimiter_images\\"+filename+'.jpg')
        cv2.imwrite(full_path, img)
                            
        return img

    def add_delimiters_images(self):
        text_output = []
        for image in self.process_images:
            filename = self.split_filename(image)
            self.add_delimiters(image, filename)
            delimiter_image = os.path.join(self.path_,'delimiter_images\\'+filename+'.jpg')
            output_file = os.path.join(self.path_, 'delimiter_images_text\\{}'.format(filename))
            f = self.get_parsed_output(delimiter_image, output_file)
            text_output.extend(f)
        return text_output
    
# print(os.getcwd())
    
path_ = os.getcwd()
# print(path_)
preprocess_images = []
for file in os.listdir("C:\\air_ticket\\comakeit_groundtravel\\preprocess_images"):
    full_path = os.path.join("C:\\air_ticket\\comakeit_groundtravel\\preprocess_images", file)
    preprocess_images.append(full_path)
print(preprocess_images)
obj = DelimiterComakeitGroundTravel(path_, preprocess_images)
text_output = obj.add_delimiters_images()


['C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FB2324-007951_page0.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FB2324-007951_page0_convert_to_rgb-set_dpi.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FB2324-007951_page1.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FB2324-007951_page1_convert_to_rgb-set_dpi.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FC2223-052798_page0.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FC2223-052798_page0_convert_to_rgb-set_dpi.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FC2223-052798_page1.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FC2223-052798_page1_convert_to_rgb-set_dpi.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FC2223-052799_page0.jpg', 'C:\\air_ticket\\comakeit_groundtravel\\preprocess_images\\FC2223-052799_page0_convert_to_rgb-set_dpi.jpg', 'C:\\air_ticket\\comakeit_groundtr

In [20]:
path_

'c:\\air_ticket\\comakeit_groundtravel'

In [142]:
import re
import glob
from prettyprinter import pprint
import pandas as pd
import os
import datetime
# from .delimiter_comakeit_groundtravel import DelimiterComakeitGroundTravel
import json

class ComakeitGroundTravel:

    def __init__(self, images):
        self.process_images = images
        # self.path_ = os.path.join(os.getcwd(), "ground_travel")
        self.path_ = os.getcwd()
        # self.path_ = "/var/www/html/image-processing-deployed/modules/documents/businesstravel"
        # self.path_ = 'modules/documents/businesstravel'

    def get_traveler_name(self, text):
        traveler_name = []
        for line in text:
            if 'in words' in line.lower() or 'duty #' in line.lower():
                break
            if 'passengers' in line.lower():
                words = line.split()
                t_name = ' '.join(words[1:])
                traveler_name.append(t_name)
        return traveler_name

    def get_dates(self, text, n):
        s_date = []
        start_dates = []
        e_date = []
        end_dates = []
        # print("\n N --> ", n)
        if n == 1:
            for line in text:
                if re.search('.*from.*to.*', line.lower()):
                    words = line.lower().split()
                    ind = words.index('from')
                    try:
                        date = datetime.datetime.strptime(words[ind+1], '%d-%m-%Y').strftime('%Y-%m-%d')
                        s_date.append(date)
                        # start_date.append(s_date)
                    except:
                        pass
                    ind = words.index('to')
                    try:
                        date = datetime.datetime.strptime(words[ind+1], '%d-%m-%Y').strftime('%Y-%m-%d')
                        e_date.append(date)
                        # end_date.append(e_date)
                    except:
                        pass
                    
                    break
                if re.search('.*booked by.*date.*', line.lower()) and 's.date e.date s.time e.time' not in line.lower() and 'veh.num. s.date e.date' not in line.lower() and 's.date e.date s.time' not in line.lower():
                    words = line.lower().split()
                    # print("WORDS --> ", words)
                    ind = words.index('date:')
                    try:
                        date = datetime.datetime.strptime(words[ind+1], '%d-%m-%Y').strftime('%Y-%m-%d')
                        s_date.append(date)
                        # start_date.append(s_date)
                        e_date.append(date)
                        # end_date.append(e_date)
                    except:
                        pass
                    
                    break

            return s_date, e_date

        else:
            flag = 0
            year = None
            for i in range(len(text)):
                if 's.date e.date s.time e.time' in text[i].lower():
                    for j in range(len(text)):
                        pattern = re.compile(('[0-9][0-9]-[0-9][0-9]-\d\d\d\d'))
                        date = pattern.search(text[j])
                        if date:
                            date = date.group()
                            year = date.split("-")[-1]
                            # print("YEAR --> ", year)
                            break 
                    if year:
                        for j in range(i+1, len(text)):
                            pattern = ('[0-9][0-9]-[0-9][0-9]')
                            p = re.compile(pattern)
                            result = p.findall(text[j])
                            if result:
                                # print('RESULT----> ', result)
                                result[0] = result[0] + "-" + year 
                                result[1] = result[1] + "-" + year
                                try:
                                    date = datetime.datetime.strptime(result[0], '%d-%m-%Y').strftime('%Y-%m-%d')
                                    start_dates.append(date)
                                except:
                                    pass
                                try:
                                    date = datetime.datetime.strptime(result[1], '%d-%m-%Y').strftime('%Y-%m-%d')
                                    end_dates.append(date)
                                except:
                                    pass
                                if 'total' in text[j].lower():
                                    flag = 1
                                    break
                if flag == 1:
                    break

            return start_dates, end_dates

    def get_distance(self, text, n):
        d = []
        # print("\n TRAVELLERS --> ", n)
        if n == 1 or n == 0:
            for i in range(len(text)):
                if 'start end total extra' in text[i].lower():
                    words = text[i+1].split()
                    dis = words[-2]
                    # print("HI", text[i].lower(), dis)
                    d.append(dis)
        else:
            flag = 0
            for i in range(len(text)):
                if 's.date e.date s.time e.time' in text[i].lower():
                    # print("HELLO", text[i].lower())
                    for j in range(i+1, len(text)):
                        words = text[j].split()
                        if len(words) > 6:
                            dis = words[-7]
                            d.append(dis)
                        if 'total' in text[j].lower():
                            flag = 1
                            break
                if flag == 1:
                    break
        return d

    def get_distance_unit(self, text):
        dis_unit = []
        for i in range(len(text)):
            if re.search('.*start.*end.*total.*extra.*', text[i].lower()) or re.search('.*start.*end.*', text[i].lower()):
                unit = text[i+1].split()[0]
                if not unit.isnumeric():
                    dis_unit.append(unit)
        # print(dis_unit)
        return dis_unit

    def is_float(self, string):
        try:
            float(string)
            return True
        except ValueError:
            return False

    def get_additional_charges(self, text):
        charges = []
        flag = 0
        debarred = ['total', 'for', 'sor']

        for i in range(len(text)):
            if 'passenger veh.num. s.date e.date' in text[i].lower():
                for j in range(i+1, len(text)):
                    words = text[j].split()
                    # print("WORDS --> ", words)
                    if 'total' in text[j].lower():
                        flag = 1
                        break
                    if self.is_float(words[-1]) or words[-1].isnumeric():
                        if '.' in words[-1] and len(words[-1].split('.')[-1]) > 0:
                            charges.append(words[-1])
            if flag == 1:
                break
        
        return charges
    
    def get_stationary_asset(self, text):

        asset = []

        data = {
            'first': 'Dendukuri House',
            'second': 'Galaxy Aurobindo 9th Floor',
            'third': 'Galaxy Aurobindo 5th Floor'
        }

        for line in text:
            if re.search('.*plot no 564/a39.*phase.*road no 92.*', line.lower()):
                print(1)
                asset.append(data['first'])
                break
            elif re.search('.*7th.*floor.*block.*', line.lower()) or re.search('.*9th.*floor.*galaxy.*', line.lower()) or re.search('.*aurobindo.*galaxy.*9th.*floor.*', line.lower()):
                print(2)
                asset.append(data['second'])
                break
            elif re.search('.*plot.*no.*564.*a.*39.*', line.lower()):
                print(3)
                asset.append(data['first'])
                break
            elif re.search('.*phase.*road no. 92.*plot no.*', line.lower()):
                print(4)
                asset.append(data['first'])
                break
            elif re.search('.*9th.*aurobindo galaxy.*', line.lower()):
                print(5)
                asset.append(data['second'])
                break
            elif re.search('.*9th.*aurabindo.*', line.lower()):
                print(6)
                asset.append(data['second'])
                break
            elif re.search('.*aurobindo.*9th.*', line.lower()):
                print(7)
                asset.append(data['second'])
                break
            elif re.search('.*5th.*aurobindo.*', line.lower()):
                print(8)
                asset.append(data['third'])
                break

        # if not asset:
            # print('hello hello')
            # asset.append('')
        print(asset)
        return asset

    # def get_stationary_asset(self, text):

    #     asset = []

    #     data = {
    #         'first': 'Dendukuri House',
    #         'second': 'Galaxy Aurobindo 9th Floor',
    #         'third': 'Galaxy Aurobindo 5th Floor'
    #     }

    #     for line in text:
    #         if re.search('.*plot no 564/a39.*phase.*road no 92.*', line.lower()):
    #             asset.append(data['first'])
    #             break

    #     return asset

    def get_trip_costs(self, text):
        costs = []
        flag = 0
        amt = 0
        # flag2 = 0

        for i in range(len(text)):
            if 'sr. description rate qty amount' in text[i].lower():
                # flag2 = 1
                for j in range(i+1, len(text)):
                    if '#' in text[j]:
                        for k in range(j+1, len(text)):
                            words = text[k].split()
                            if "extra km" in text[k].lower():
                                # print("INSIDE")
                                val = text[k].split()[-1]
                                # print("\n INSIDE VALUE --> ", val)
                                if val.replace(",", "").replace(".", "").isnumeric():
                                    val = val.replace(",", "")
                                    if costs:
                                        costs[0] = float(costs[0]) + float(val)
                                        # print(costs[0], val)
                                    else:
                                        costs.append(val)
                                break
                            if 'extra hours' in text[k].lower() or 'total' in text[k].lower():
                                # costs.append(amt)
                                flag = 1
                                break
                            if self.is_float(words[-1].replace(',', '')):
                                if '.' in words[-1] and len(words[-1].split('.')[-1]) > 0 and 'total' not in text[k].lower():
                                    # print(words[-1])
                                    # amt += float(words[-1].replace(',', ''))
                                    amt = float(words[-1].replace(',', ''))
                                    costs.append(amt)
                                    # break
                                    # print(amt)
                            elif '#' in text[k]:
                                costs.append(amt)
                                break
                            # if 'total' in text[k].lower():
                            #     costs.append(amt)
                            #     flag = 1
                            #     break
                    if flag == 1:
                        break
                if flag == 1:
                    break
        return costs

    def get_expense_type(self, text):
        return ['Taxi']

    def get_currency_code(self, text):
        return ['INR']

    def get_bill_no(self, text):
        inv_num = []
        for line in text:
            if 'invoice number' in line.lower():
                inv_num.append(line.split()[-1])
                break
        return inv_num

    def ensemble_output_files(self, text_output):
        ensemble_output_files = []
        names = []
        num = 0
        for file in text_output:
            with open(file, encoding='utf-8') as fp:
                print("\n FILE --> ", file)
                text = fp.read()
                text = text.splitlines()
                text = [line for line in text if not line.isspace() and len(line) > 0]

                filename = os.path.split(file)[-1].split('.txt')[0]
                ind = filename.index('page')
                pre = filename[ind+5:]
                filename = filename[:ind+5]

                asset_names = self.get_stationary_asset(text)
                # print('asset names---> ', asset_names)
                traveler_names = self.get_traveler_name(text)
                if num == 0:
                    names = traveler_names
                n = len(names)
                expense_types = self.get_expense_type(text)
                start_dates = self.get_dates(text, n)[0]
                end_dates = self.get_dates(text, n)[1]
                distances = self.get_distance(text, n)
                distance_units = self.get_distance_unit(text)
                trip_costs = self.get_trip_costs(text)
                currency_codes = self.get_currency_code(text)
                additional_charges = self.get_additional_charges(text)
                bill_nos = self.get_bill_no(text)

                text_file = os.path.join(self.path_, 'ensemble_output/{}.txt'.format(filename))
                if text_file not in ensemble_output_files:
                    ensemble_output_files.append(text_file)
                with open(text_file, 'a', encoding='utf-8') as fp:
                    fp.write(str(asset_names)+"\n")
                    fp.write(str(traveler_names)+"\n")
                    fp.write(str(expense_types)+"\n")
                    fp.write(str(start_dates)+"\n")
                    fp.write(str(end_dates)+"\n")
                    fp.write(str(distances)+"\n")
                    fp.write(str(distance_units)+"\n")
                    fp.write(str(trip_costs)+"\n")
                    fp.write(str(currency_codes)+"\n")
                    fp.write(str(additional_charges)+"\n")
                    fp.write(str(bill_nos)+"\n")
                    fp.write('')
                    fp.write(str(filename)+"\n")
                    fp.write('---\n')
                num += 1
        return ensemble_output_files

    def get_result(self, text_output):
        # obj = DelimiterComakeitGroundTravel(self.path_, self.process_images)
        # text_output = obj.add_delimiters_images()
        ensemble_output_files = self.ensemble_output_files(text_output)
        print("OUTPUT FILES --> ", ensemble_output_files)
        result = {}
        for e_file in ensemble_output_files:
            asset_names_ex = []
            traveler_names_ex = []
            expense_types_ex = []
            start_dates_ex = []
            end_dates_ex = []
            distances_ex = []
            distance_units_ex = []
            trip_costs_ex = []
            currency_codes_ex = []
            additional_charges_ex = []
            bill_nos_ex = []
            stationary_asset_final = []
            traveler_names_final = []
            expense_types_final = []
            start_dates_final = []
            end_dates_final = []
            distances_final = []
            distance_units_final = []
            trip_costs_final = []
            currency_codes_final = []
            additional_charges_final = []
            bill_numbers_final = []
            filename = os.path.split(e_file)[-1].split('.txt')[0]
            with open(e_file) as fp:
                outputs = fp.read().split("---\n")[:-1]
                for out in outputs:
                    data = out.splitlines()
                    asset_names = eval(data[0])
                    traveler_names = eval(data[1])
                    expense_types = eval(data[2])
                    start_dates = eval(data[3])
                    end_dates = eval(data[4])
                    distances = eval(data[5])
                    distance_units = eval(data[6])
                    trip_costs = eval(data[7])
                    currency_codes = eval(data[8])
                    additional_charges = eval(data[9])
                    bill_nos = eval(data[10])

                    if traveler_names and not traveler_names_final:
                        traveler_names_final = traveler_names

                    n = len(traveler_names_final)

                    if expense_types and not expense_types_final:
                        expense_types_final = expense_types
                        
                    if start_dates and not start_dates_final:
                        start_dates_final = start_dates

                    if end_dates and not end_dates_final:
                        end_dates_final = end_dates

                    if distances and not distances_final:
                        distances_final = distances

                    if distance_units and not distance_units_final:
                        distance_units_final = distance_units

                    if additional_charges and not additional_charges_final:
                        additional_charges_final = additional_charges

                    if asset_names and not stationary_asset_final:
                        stationary_asset_final = asset_names

                    if trip_costs and not trip_costs_final:
                        trip_costs_final = trip_costs

                    if currency_codes and not currency_codes_final:
                        currency_codes_final = currency_codes

                    if bill_nos and not bill_numbers_final:
                        bill_numbers_final = bill_nos

            traveler_names_ex.extend(traveler_names_final)
            asset_names_ex.extend(stationary_asset_final)
            expense_types_ex.extend(expense_types_final)
            start_dates_ex.extend(start_dates_final)
            end_dates_ex.extend(end_dates_final)
            distances_ex.extend(distances_final)
            bill_nos_ex.extend(bill_numbers_final)
            currency_codes_ex.extend(currency_codes_final)
            additional_charges_ex.extend(additional_charges_final)
            distance_units_ex.extend(distance_units_final)
            trip_costs_ex.extend(trip_costs_final)
            # print(traveler_names_final, traveler_names_ex)

            asset_names_ex = asset_names_ex*len(traveler_names_ex)
            # expense_types_ex = expense_types_ex*len(traveler_names_ex)
            bill_nos_ex = bill_nos_ex*len(traveler_names_ex)
            if not distances_ex:
                    distances_ex.extend([None]*len(traveler_names_ex))
            if not distance_units_ex:
                distance_units_ex.extend([None]*len(traveler_names_ex))
            if not trip_costs_ex:
                trip_costs_ex.extend([None]*len(traveler_names_ex))
            if not additional_charges_ex:
                additional_charges_ex.extend([None]*len(traveler_names_ex))
            # print(traveler_names_ex)
            # print(asset_names_ex)
            # print(start_dates_ex)
            # print(end_dates_ex)
            final_output_file = os.path.join(self.path_, 'final_ensemble_output/{}.txt'.format(filename))
            with open(final_output_file, 'w') as fp:
                fp.write(str(asset_names_ex)+"\n")
                fp.write(str(traveler_names_ex)+"\n")
                fp.write(str(expense_types_ex)+"\n")
                fp.write(str(start_dates_ex)+"\n")
                fp.write(str(end_dates_ex)+"\n")
                fp.write(str(distances_ex)+"\n")
                fp.write(str(distance_units_ex)+"\n")
                fp.write(str(trip_costs_ex)+"\n")
                fp.write(str(currency_codes_ex)+"\n")
                fp.write(str(additional_charges_ex)+"\n")
                fp.write(str(bill_nos_ex)+"\n")
        # all_data = []
        # for a, t, e, sd, ed, de, due, tc, cc, ac, bn in zip(asset_names_ex, traveler_names_ex, expense_types_ex, start_dates_ex, 
        #                                                     end_dates_ex, distances_ex, distance_units_ex, trip_costs_ex, currency_codes_ex, additional_charges_ex, bill_nos_ex):
        #     json_out = {
        #         'stationary_asset': a,
        #         'traveler_name': t,
        #         'record_type': 'ground_travel',
        #         'expense_type': e,
        #         'start_date': sd,
        #         'end_date': ed,
        #         'distance': de,
        #         'distance_units': due,
        #         'trip_cost': tc,
        #         'currency_code': cc,
        #         'additional_charges': ac,
        #         'bill_no': bn
        #     }        
        #     all_data.append(json_out)
        # json_file = os.path.join(self.path_, f"json_output/{filename}.json")
        # with open(json_file, 'w') as fp:
        #     json.dump(all_data, fp)
        # all_pages = [{'lineitems': all_data}]
        # return all_pages
            # all_pages.append({"lineitems": all_data})
        # result["record_type"] = "ground_travel"
        # result["asset_name"] = asset_names_ex
        # result["traveler_names"] = 


obj = ComakeitGroundTravel(preprocess_images)
obj.get_result(text_output)



 FILE -->  c:\air_ticket\comakeit_groundtravel\delimiter_images_text\FB2324-007951_page0_4.txt
8
['Galaxy Aurobindo 5th Floor']

 FILE -->  c:\air_ticket\comakeit_groundtravel\delimiter_images_text\FB2324-007951_page0_convert_to_rgb-set_dpi_4.txt
8
['Galaxy Aurobindo 5th Floor']

 FILE -->  c:\air_ticket\comakeit_groundtravel\delimiter_images_text\FB2324-007951_page1_4.txt
[]

 FILE -->  c:\air_ticket\comakeit_groundtravel\delimiter_images_text\FB2324-007951_page1_convert_to_rgb-set_dpi_4.txt
[]

 FILE -->  c:\air_ticket\comakeit_groundtravel\delimiter_images_text\FC2223-052798_page0_4.txt
1
['Dendukuri House']

 FILE -->  c:\air_ticket\comakeit_groundtravel\delimiter_images_text\FC2223-052798_page0_convert_to_rgb-set_dpi_4.txt
1
['Dendukuri House']

 FILE -->  c:\air_ticket\comakeit_groundtravel\delimiter_images_text\FC2223-052798_page1_4.txt
[]

 FILE -->  c:\air_ticket\comakeit_groundtravel\delimiter_images_text\FC2223-052798_page1_convert_to_rgb-set_dpi_4.txt
[]

 FILE -->  c:\air

In [143]:
s = set()
for file in os.listdir("C:\\air_ticket\\comakeit_groundtravel\\final_ensemble_output\\"):
    s.add(file.split("_")[0])

In [144]:
len(s)

130

In [145]:
# filenames = []
outputs = {}
for file in os.listdir("C:\\air_ticket\\comakeit_groundtravel\\final_ensemble_output\\"):
    full_path = os.path.join("C:\\air_ticket\\comakeit_groundtravel\\final_ensemble_output\\", file)
    with open(full_path) as fp:
        data = fp.read().split("\n")
        # print(data)
        value = {
            'asset_name' : eval(data[0]),
            'traveler_name': eval(data[1]),
            'expense_type': eval(data[2]),
            'start_date' : eval(data[3]),
            'end_date': eval(data[4]),
            'dis' : eval(data[5]),
            'dis_unit' : eval(data[6]),
            'trip_cost' : eval(data[7]),
            'currency_code' : eval(data[8]),
            'add_charge' : eval(data[9]),
            'bill_no' : eval(data[10])
        }
        short_name = file.split("_")[0]
        if short_name in outputs:
            prev = outputs[short_name]
            for i, j in value.items():
                if value[i] and prev[i]:
                    if not prev[i][0] and value[i][0]:
                        prev[i][0] = value[i][0]
            outputs[short_name] = prev
        else:
            outputs[short_name] = value 
        
    

In [151]:
# outputs['FC2223-052799']
# print(outputs)

for k,v in outputs.items():
    if not v['asset_name']:
        print(outputs[k])

In [152]:
len(outputs)

130

In [153]:
df = pd.DataFrame(outputs)

In [154]:
df = df.T

In [155]:
df.index = range(len(df))

In [156]:
for col in df.columns:
    # if df[col][0]:
    # print(df[col])
        df[col] = df[col].apply(lambda x: x[0])
    # else:
        # df[col] = ''

In [157]:
df

,asset_name,traveler_name,expense_type,start_date,end_date,dis,dis_unit,trip_cost,currency_code,add_charge,bill_no
0,Galaxy Aurobindo 5th Floor,Kiran Madhunapantula.,Taxi,2023-11-13,2023-11-13,None,KM,3910.0,INR,170.00,FB2324-007951
1,Dendukuri House,Balaji Tunuguntla,Taxi,2022-12-03,2022-12-04,66,KM,1700.0,INR,0.00,FC2223-052798
2,Dendukuri House,Balaji Tunuguntla,Taxi,2022-12-11,2022-12-11,140,KM,2720.0,INR,290.00,FC2223-052799
3,Dendukuri House,Sanjay Akondi,Taxi,2022-12-09,2022-12-09,90,KM,1870.0,INR,250.00,FC2223-052801
4,Dendukuri House,Shiva Prasad Kamini,Taxi,2022-12-09,2022-12-09,80,KM,1700.0,INR,250.00,FC2223-052803
...,...,...,...,...,...,...,...,...,...,...,...
125,Galaxy Aurobindo 5th Floor,Deepthi,Taxi,2023-11-27,2023-11-27,143,KM,2771.0,INR,390.00,FC2324-059453
126,Galaxy Aurobindo 5th Floor,Kiran Madhunapantula.,Taxi,2023-10-27,2023-10-27,102,KM,3940.0,INR,350.00,FD2324-009156
127,Galaxy Aurobindo 5th Floor,Mr. BARRY,Taxi,2023-10-29,2023-10-29,90,KM,3580.0,INR,170.00,FD2324-009162
128,Dendukuri House,Steven Ten Napel,Taxi,2022-12-14,2022-12-14,151,KM,2130.00,INR,600.00,GR2223-002773


In [158]:
df.to_excel("groundtravel_comakeit3.xlsx")

In [175]:
df = pd.read_excel('groundtravel_comakeit_merged.xlsx')
df

,asset_name,traveler_name,expense_type,start_date,end_date,dis,dis_unit,trip_cost,currency_code,add_charge,bill_no
0,Dendukuri House,Balaji Tunuguntla,Taxi,2022-12-03,2022-12-04,66,KM,1700.0,INR,0.0,FC2223-052798
1,Dendukuri House,Balaji Tunuguntla,Taxi,2022-12-11,2022-12-11,140,KM,2720.0,INR,290.0,FC2223-052799
2,Dendukuri House,Sanjay Akondi,Taxi,2022-12-09,2022-12-09,90,KM,1870.0,INR,250.0,FC2223-052801
3,Dendukuri House,Shiva Prasad Kamini,Taxi,2022-12-09,2022-12-09,80,KM,1700.0,INR,250.0,FC2223-052803
4,Dendukuri House,Shiva Prasad Kamini,Taxi,2022-12-09,2022-12-09,85,KM,1785.0,INR,0.0,FC2223-052806
...,...,...,...,...,...,...,...,...,...,...,...
219,Galaxy Aurobindo 5th Floor,Deepthi,Taxi,2023-11-27,2023-11-27,143,KM,2771.0,INR,390.0,FC2324-059453
220,Galaxy Aurobindo 5th Floor,Kiran Madhunapantula.,Taxi,2023-10-27,2023-10-27,102,KM,3940.0,INR,350.0,FD2324-009156
221,Galaxy Aurobindo 5th Floor,Mr. BARRY,Taxi,2023-10-29,2023-10-29,90,KM,3580.0,INR,170.0,FD2324-009162
222,Dendukuri House,Steven Ten Napel,Taxi,2022-12-14,2022-12-14,151,KM,2130.0,INR,600.0,GR2223-002773


In [173]:
dup_df = df[df.duplicated(subset=['traveler_name',	'expense_type',	'start_date',	'end_date',	'dis',	'dis_unit',	'trip_cost',	'currency_code',	'add_charge',	'bill_no'
])]

In [44]:
data = ["['hi']", "['hello']", "['bye']"]
print(eval(data[0]))

['hi']


In [174]:
dup_df.to_excel('duplicates_merged.xlsx')

In [176]:
result_df = df.drop_duplicates(subset=['traveler_name',	'expense_type',	'start_date',	'end_date',	'dis',	'dis_unit',	'trip_cost',	'currency_code',	'add_charge',	'bill_no'], keep=False)
len(result_df)

42

In [177]:
result_df.to_excel('groundtravel_comakeit_final.xlsx')